In [1]:
import numpy as np
from tqdm import tqdm
import random as rd

In [2]:
RBP='1A34 1AQ3 1BMV 1F7V 1H4Q 1JBT 1L9A 1MZP 1WMQ 1YYK 1ZL3 2ANN 2ATW 2AZ2 2BX2 2DR2 2DU4 2G4B 2GXB 3AMT'
#the rbp we gonna analyze
rbp=RBP.split()
len(rbp)

20

In [3]:
name=[]
for i in range(20):
    f = open(rbp[i]+'/complex1.pdb')
    for x in f.readlines():
        if x.split()[3] not in name:
            name.append(x.split()[3])
    f.close()
len(name)
#save all the elements from the ebp
#in fact, after 24 there is no useful elements, just to prevent bugs.

38

In [4]:
protein = name[0:20]
protein.remove('ASER')
protein.remove('AARG')
protein.append('HIS')
protein.append('CYS')
protein.extend(['A','G','C','U' ])

In [5]:
num = 24
comp_num = 100
total = 20*comp_num

In [6]:
#measure the size of the RBP

size=np.zeros([20,comp_num,6])
for i in tqdm(range(20)):#for every rbp
    for j in range(comp_num):
        r=rbp[i]
        f = open(r+'/complex%d.pdb'%(j+1))
        tmp_x = []
        tmp_y = []
        tmp_z = []
        for x in f.readlines():
            tmp_z.append(np.float32(x[-9:-1]))
            tmp_y.append(np.float32(x[-17:-9]))
            tmp_x.append(np.float32(x[-25:-17]))
        size[i,j,0]=max(tmp_x)
        size[i,j,3]=min(tmp_x)
        size[i,j,1]=max(tmp_y)
        size[i,j,4]=min(tmp_y)
        size[i,j,2]=max(tmp_z)
        size[i,j,5]=min(tmp_z)


100%|██████████| 20/20 [00:46<00:00,  2.24s/it]


In [7]:
length=np.zeros([20,comp_num,3])
length[:,:,0]=(size[:,:,0]-size[:,:,3])/num
length[:,:,1]=(size[:,:,1]-size[:,:,4])/num
length[:,:,2]=(size[:,:,2]-size[:,:,5])/num
coordinate=np.zeros((20,comp_num,num,num,num,24), dtype=np.int)
for k in tqdm(range(20)):#for every rbp
    for j in range(comp_num):
        r=rbp[k]
        f = open(r+'/complex%d.pdb'%(j+1))
        t=f.readlines()[0].split()[-4]
        cotmp=np.zeros(4)
        f = open(r+'/complex%d.pdb'%(j+1))
        for x in f.readlines():
            #print(x)
            if x[22:26]!=t and cotmp[3]!=0:
                #print(cotmp,t,tmp)
                position = cotmp[0:3]/cotmp[3]-size[k,j,3:]
                #print(cotmp[0:3]/cotmp[3])
                ix=position[0]//length[k,j,0]
                iy=position[1]//length[k,j,1]
                iz=position[2]//length[k,j,2]
                #if ix > 10:
                    #print(k,j,position)
                if i < 24:
                    coordinate[k,j,int(ix),int(iy),int(iz),i]+=1
                cotmp=np.zeros(4)
            #save center of the acid or RNA in coord

            tmp=np.zeros(4)
            tmp[2]+=np.float32(x[-9:-1])
            tmp[1]+=np.float32(x[-17:-9])
            tmp[0]+=np.float32(x[-25:-17])
            tmp[3]+=1
            #print(tmp)
            cotmp+=tmp

            #count every atom in each acid or RNA

            n =x.split()[3]
            i = name.index(n)     
            t =x[22:26]
            #update the tmp index 

    f.close() 

100%|██████████| 20/20 [01:15<00:00,  3.64s/it]


In [8]:
index = list(range(total))
rd.shuffle(index)
train_index = index[:-300]
test_index = index[-300:]

In [9]:
train_coordinate = coordinate.reshape(20*comp_num,num,num,num,24)[train_index]
test_coordinate = coordinate.reshape(20*comp_num,num,num,num,24)[test_index]

In [10]:
target = np.zeros((20,comp_num))

for k in range(20):
    tmp=[]
    f=open(rbp[k]+'/'+rbp[k]+'.rmsd.out.tensorflow')
    for x in f.readlines()[1:]:
        tmp.append(np.float32(x[23:32]))
    target[k]=np.float32(tmp[0:comp_num])

In [11]:
train_target = target.reshape(20*comp_num,1)[train_index]
test_target = target.reshape(20*comp_num,1)[test_index]

In [12]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [13]:
x = tf.placeholder(tf.float32, shape=[None,  num, num, num, 24], name='x_')
y_ = tf.placeholder(tf.float32, shape=[None, 1], name='y_')

In [14]:
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1, 1], padding='SAME')

def max_pool_2x2x2(x):
    return tf.nn.max_pool3d(x, ksize=[1, 2, 2, 2, 1],
                        strides=[1, 2, 2, 2, 1], padding='SAME')
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def random_batch(data,label,size):
    index =  np.random.choice(len(label), size, replace=False)
    batch_data = data[index]
    batch_label = label[index]
    return batch_data,batch_label

In [15]:
x_image = tf.cast(tf.reshape(x, [-1, num, num, num, 24]),tf.float32)

In [16]:
W_conv1 = weight_variable([3, 3, 3, 24, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv3d(x_image, W_conv1) + b_conv1)

W_conv2 = weight_variable([6, 6, 6, 32, 32])
b_conv2 = bias_variable([32])
h_conv2 = tf.nn.relu(conv3d(h_conv1, W_conv2) + b_conv2)

h_pool1 = max_pool_2x2x2(h_conv1)

In [17]:
W_conv3 = weight_variable([4, 4, 4, 32, 32])
b_conv3 = bias_variable([32])
h_conv3 = tf.nn.relu(conv3d(h_pool1, W_conv3) + b_conv3)

W_conv4 = weight_variable([5, 5, 5, 32, 32])
b_conv4 = bias_variable([32])
h_conv4 = tf.nn.relu(conv3d(h_conv3, W_conv4) + b_conv4)

h_pool2 = max_pool_2x2x2(h_conv4)

In [18]:
W_fc1 = weight_variable([int(num* num *num/2), 256])
b_fc1 = bias_variable([256])

h_pool2_flat = tf.reshape(h_pool2, [-1, int(num * num* num /2)])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [19]:
keep_prob = tf.placeholder(tf.float32, name='keep_prob0')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [20]:
W_fc2 = weight_variable([256, 1])
b_fc2 = bias_variable([1])

y_conv = tf.add(tf.matmul(h_fc1_drop, W_fc2) , b_fc2,name='y_conv')

In [21]:
loss = tf.reduce_mean(tf.squared_difference(y_,y_conv), name='loss')
train_step = tf.train.AdamOptimizer(5e-5).minimize(loss)
saver = tf.train.Saver()

size = 30

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
        batch = random_batch(train_coordinate,train_target,size)
        test_batch = random_batch(test_coordinate,test_target,50)
        if i % 30 == 0:
            #train_accuracy = accuracy.eval(feed_dict={
                  #x: np.float32(coordinate), y_: np.float32(target), keep_prob: 1.0})
            print('step %d, test loss %g' % (i, loss.eval(feed_dict={
                  x: test_batch[0], y_: test_batch[1], keep_prob: 1.0})))
        if i % 500 == 0:    
            saver.save(sess, './my-model6')
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})


step 0, test loss 721.686
step 30, test loss 401.073
step 60, test loss 363.316
step 90, test loss 238.588
step 120, test loss 380.644
step 150, test loss 271.923
step 180, test loss 277.594
step 210, test loss 331.216
step 240, test loss 241.509
step 270, test loss 283.441
step 300, test loss 166.909
step 330, test loss 233.589
step 360, test loss 232.099
step 390, test loss 167.911
step 420, test loss 133.633
step 450, test loss 163.951
step 480, test loss 124.109
step 510, test loss 244.315
step 540, test loss 139.906
step 570, test loss 142.561
step 600, test loss 157.918
step 630, test loss 193.053
step 660, test loss 151.35
step 690, test loss 165.745
step 720, test loss 106.179
step 750, test loss 166.402
step 780, test loss 165.987
step 810, test loss 116.528
step 840, test loss 156.214
step 870, test loss 190.227
step 900, test loss 105.777
step 930, test loss 174.264
step 960, test loss 111.481
step 990, test loss 144.018
step 1020, test loss 153.285
step 1050, test loss 140.

In [22]:
import pickle

def test_model(test_num):
    """
    Test the saved model against the test dataset
    """
    save_model_path = './my-model6'
    #test_features, test_labels = pickle.load(open('preprocess_test.p', mode='rb'))
    loaded_graph = tf.Graph()
    compare_batch = random_batch(test_coordinate,test_target,test_num)
    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(sess, save_model_path)
    
        # Get Tensors from loaded model
        loaded_x = loaded_graph.get_tensor_by_name('x_:0')
        loaded_y = loaded_graph.get_tensor_by_name('y_:0')
        loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_prob0:0')
        loaded_loss = loaded_graph.get_tensor_by_name('loss:0')
        loaded_yconv = loaded_graph.get_tensor_by_name('y_conv:0')

        predict_y = loaded_yconv.eval(feed_dict={loaded_x: test_coordinate[:test_num],
                                       loaded_y: test_target[:test_num], loaded_keep_prob: 1.0})
        real_y = loaded_y.eval(feed_dict={loaded_x: test_coordinate[:test_num],
                                       loaded_y: test_target[:test_num], loaded_keep_prob: 1.0})       
        #print(tf.nn.top_k(conseq,10,sorted=False))
    return predict_y,real_y


In [23]:
test_num = 100
a = test_model(test_num)[0]
b = test_model(test_num)[1]
c = np.zeros([test_num,2])
c[:,0]=np.reshape(a,[test_num])
c[:,1]=np.reshape(b,[test_num])
c[np.argsort(c[:,0])]

INFO:tensorflow:Restoring parameters from ./my-model6
INFO:tensorflow:Restoring parameters from ./my-model6


array([[   7.2552886 ,   24.47680092],
       [   8.01174831,    4.78395987],
       [   8.68395519,   13.30039978],
       [  11.80395508,    4.08218002],
       [  11.80395889,    6.9609499 ],
       [  12.31445503,   19.05990028],
       [  12.95008087,    1.75127006],
       [  16.00240135,    6.27271986],
       [  17.70624733,   18.69039917],
       [  18.49664688,   22.35589981],
       [  18.62885284,   19.15099907],
       [  18.88774872,   11.15680027],
       [  18.88774872,   19.91360092],
       [  18.88774872,   25.65990067],
       [  19.17640305,   20.08370018],
       [  19.80217934,   26.16379929],
       [  20.56017494,   21.37619972],
       [  21.44882202,   27.58270073],
       [  22.16014671,   16.85519981],
       [  22.59461212,   17.3512001 ],
       [  22.83474159,   29.77980042],
       [  23.45298386,   18.48920059],
       [  24.14907646,   22.85250092],
       [  24.38992691,   24.41469955],
       [  25.28885269,   18.93919945],
       [  25.96772957,   